In [2]:
%%capture

!pip install bs4
!pip install nbformat
!pip install pandas
!pip install geopandas
!pip install requests
!pip install --upgrade xlrd

In [3]:
%%capture

%run ./Tidy.ipynb

# Phase de modélisation

On entre ici provisoirement dans une phase de test ou l'on va tester plusieurs modèle pour voir leur pertinence. 
Rien de définitif pour l'instant, on ne sait pas si il s'agira du modèle final.

### Test: faire un classifieur

On essaie de prédire la législation d'un pays (Legal, Illegal, Decriminalized -> Soit une variable discrète) à partir des données dont on dispose sur la consommation de cannabis.

In [13]:
# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie ntière pour faire un premier test juste avec ça.
import pandas as pd

df = pd.read_csv('~/work/Projet_Python_2A/Data/EUDA/edr2024-gps-current-table-1.csv')

dfcurrent = df[df['Substance'] == 'Cannabis']

dfcurrentadults = dfcurrent[dfcurrent['Age'] == 'All adults (15-64)']

dfcurrentadultslf = dfcurrentadults[dfcurrentadults['Recall period'] == 'Lifetime']

# On essaie maintenant de merge les deux tables
dfadultslf_legal = dfcurrentadultslf.join(legal, on = 'Country')

print(dfadultslf_legal)

          Country Country code  Survey year  Prevalence (%) Substance  \
0         Austria           AT         2020            22.7  Cannabis   
42        Belgium           BE         2018            22.6  Cannabis   
62       Bulgaria           BG         2020             8.7  Cannabis   
104       Croatia           HR         2019            22.9  Cannabis   
146        Cyprus           CY         2022            18.0  Cannabis   
188       Czechia           CZ         2020            29.9  Cannabis   
228       Denmark           DK         2023            37.6  Cannabis   
258       Estonia           EE         2018            24.5  Cannabis   
294       Finland           FI         2022            31.2  Cannabis   
332        France           FR         2021            47.3  Cannabis   
362       Germany           DE         2021            34.7  Cannabis   
404        Greece           GR         2015            11.0  Cannabis   
436       Hungary           HU         2019        

In [19]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Prevalence (%)','Males (%)','Females (%)']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)


# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear', random_state=2)

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

print(Y_pred)
print(Y_test)

[1 1 1 1 1 0]
42     1
0      1
516    1
332    0
721    0
871    0
Name: Recreational_num, dtype: int64


/tmp/ipykernel_76246/874937562.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_76246/874937562.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
